In [2]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append(os.path.abspath('..'))

In [3]:
data_json = pd.read_json('../outputs/preprocessed/train_data.json')
data_train = pd.read_csv('../data/MELD.Raw/train/train_sent_emo.csv')

In [4]:
data_json['filename'] = data_json['audio'].apply(lambda x: x.split("audio\\")[-1])

In [5]:
data_json.head()

,video,audio,text,label,filename
0,..\data\MELD.Raw\train\train_splits\dia0_utt0.mp4,..\data\MELD.Raw\train\audio\dia0_utt0.wav,also I was the point person on my companys tr...,neutral,dia0_utt0.wav
1,..\data\MELD.Raw\train\train_splits\dia0_utt1.mp4,..\data\MELD.Raw\train\audio\dia0_utt1.wav,You mustve had your hands full.,neutral,dia0_utt1.wav
2,..\data\MELD.Raw\train\train_splits\dia0_utt2.mp4,..\data\MELD.Raw\train\audio\dia0_utt2.wav,That I did. That I did.,neutral,dia0_utt2.wav
3,..\data\MELD.Raw\train\train_splits\dia0_utt3.mp4,..\data\MELD.Raw\train\audio\dia0_utt3.wav,So lets talk a little bit about your duties.,neutral,dia0_utt3.wav
4,..\data\MELD.Raw\train\train_splits\dia0_utt4.mp4,..\data\MELD.Raw\train\audio\dia0_utt4.wav,My duties? All right.,surprise,dia0_utt4.wav


In [6]:
data_json.shape

(9988, 5)

In [ ]:
def add_speaker(path_json, path_dataset, save_path=None):
    json_df = pd.read_json(path_json)
    dataset = pd.read_csv(path_dataset)
    json_df['filename'] = json_df['audio'].apply(lambda x: x.split("audio\\")[-1])
    for index, row in dataset.iterrows():
        filename = f"dia{row['Dialogue_ID']}_utt{row['Utterance_ID']}.wav"
        if filename in json_df['filename'].values:  
            json_df.loc[json_df['filename'] == filename, 'speaker'] = row['Speaker']

    if save_path:
        json_df.to_json(save_path)
    return json_df

train_data_json = add_speaker(path_json='../outputs/preprocessed/train_data.json', path_dataset='../data/MELD.Raw/train/train_sent_emo.csv', save_path='../outputs/preprocessed/train_data_speaker.json') 
test_data_json = add_speaker(path_json='../outputs/preprocessed/test_data.json', path_dataset='../data/MELD.Raw/test_sent_emo.csv', save_path='../outputs/preprocessed/test_data_speaker.json')
dev_data_json = add_speaker(path_json='../outputs/preprocessed/dev_data.json', path_dataset='../data/MELD.Raw/dev_sent_emo.csv', save_path='../outputs/preprocessed/dev_data_speaker.json')

In [56]:
train_data_json

,video,audio,text,label,filename,speaker
0,..\data\MELD.Raw\train\train_splits\dia0_utt0.mp4,..\data\MELD.Raw\train\audio\dia0_utt0.wav,also I was the point person on my companys tr...,neutral,dia0_utt0.wav,Chandler
1,..\data\MELD.Raw\train\train_splits\dia0_utt1.mp4,..\data\MELD.Raw\train\audio\dia0_utt1.wav,You mustve had your hands full.,neutral,dia0_utt1.wav,The Interviewer
2,..\data\MELD.Raw\train\train_splits\dia0_utt2.mp4,..\data\MELD.Raw\train\audio\dia0_utt2.wav,That I did. That I did.,neutral,dia0_utt2.wav,Chandler
3,..\data\MELD.Raw\train\train_splits\dia0_utt3.mp4,..\data\MELD.Raw\train\audio\dia0_utt3.wav,So lets talk a little bit about your duties.,neutral,dia0_utt3.wav,The Interviewer
4,..\data\MELD.Raw\train\train_splits\dia0_utt4.mp4,..\data\MELD.Raw\train\audio\dia0_utt4.wav,My duties? All right.,surprise,dia0_utt4.wav,Chandler
...,...,...,...,...,...,...
9983,..\data\MELD.Raw\train\train_splits\dia1038_ut...,..\data\MELD.Raw\train\audio\dia1038_utt13.wav,You or me?,neutral,dia1038_utt13.wav,Chandler
9984,..\data\MELD.Raw\train\train_splits\dia1038_ut...,..\data\MELD.Raw\train\audio\dia1038_utt14.wav,"I got it. Uh, Joey, women don't have Adam's ap...",neutral,dia1038_utt14.wav,Ross
9985,..\data\MELD.Raw\train\train_splits\dia1038_ut...,..\data\MELD.Raw\train\audio\dia1038_utt15.wav,"You guys are messing with me, right?",surprise,dia1038_utt15.wav,Joey
9986,..\data\MELD.Raw\train\train_splits\dia1038_ut...,..\data\MELD.Raw\train\audio\dia1038_utt16.wav,Yeah.,neutral,dia1038_utt16.wav,All


**Load tensors data**

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


data = torch.load(os.path.join("..", "outputs", "embeddings", "loaders_datasets.pt"))
data

{'train': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a8a25eca0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d760>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d3d0>,
  'labels': tensor([0, 0, 0,  ..., 6, 0, 1])},
 'val': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d6d0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d490>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d730>,
  'labels': tensor([4, 6, 0,  ..., 4, 4, 4])},
 'test': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d4c0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a8461c790>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a8449f7c0>,
  'labels': tensor([6, 2, 0,  ..., 0, 0, 0])}}

**Embedding of Speakers**

In [86]:
import torch
import pandas as pd

# train ../outputs/preprocessed/train_data_speaker.json
# dev ../outputs/preprocessed/dev_data_speaker.json
# test ../outputs/preprocessed/test_data_speaker.json


def embedding_speaker(json_path, data, split=None):
    json_df = pd.read_json(json_path)
    if split in data:
        speakers_to_idx = {speaker: i for i, speaker in enumerate(json_df['speaker'].unique())}
        speaker_labels = [speakers_to_idx[speaker] for speaker in json_df['speaker']]
        data[split]['speaker'] = torch.tensor(speaker_labels, dtype=torch.long)

    return data

data = embedding_speaker(json_path='../outputs/preprocessed/train_data_speaker.json', data=data, split='train')
data = embedding_speaker(json_path='../outputs/preprocessed/dev_data_speaker.json', data=data, split='val')
data = embedding_speaker(json_path='../outputs/preprocessed/test_data_speaker.json', data=data, split='test')

torch.save(data, os.path.join("..", "outputs", "embeddings", "loaders_datasets_speakers.pt"))

In [87]:
data

{'train': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a8a25eca0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d760>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d3d0>,
  'labels': tensor([0, 0, 0,  ..., 6, 0, 1]),
  'speaker': tensor([ 0,  1,  0,  ...,  2, 61,  2])},
 'val': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d6d0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d490>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d730>,
  'labels': tensor([4, 6, 0,  ..., 4, 4, 4]),
  'speaker': tensor([0, 1, 2,  ..., 1, 6, 1])},
 'test': {'audio': <torch.utils.data.dataset.TensorDataset at 0x24a83c3d4c0>,
  'text': <torch.utils.data.dataset.TensorDataset at 0x24a8461c790>,
  'video': <torch.utils.data.dataset.TensorDataset at 0x24a8449f7c0>,
  'labels': tensor([6, 2, 0,  ..., 0, 0, 0]),
  'speaker': tensor([0, 1, 1,  ..., 1, 3, 6])}}

In [88]:
data['train']['speaker'].shape

torch.Size([9988])

In [89]:
import torch
print(torch.version.cuda)  # Affiche la version de CUDA utilisée par PyTorch
print(torch.cuda.is_available())  # Vérifie si CUDA est bien accessible


11.3
True


In [93]:
df_json = pd.read_json('../outputs/preprocessed/train_data_speaker.json')
len(df_json['speaker'].unique())

260

In [15]:
import sys, os
sys.path.append(os.path.abspath('..'))

data = torch.load(os.path.join("..", "outputs", "embeddings", "loaders_datasets_speakers.pt"))


In [26]:
data['train']['audio'].tensors[0].shape

torch.Size([9988, 768])